In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.datasets import make_circles
import matplotlib.pyplot as plt
import torch

In [ ]:
x,y = make_circles(n_samples=1000000, noise=0.05, random_state=40)

In [ ]:
circles = pd.DataFrame({"X1":x[:,0],
                        "X2":x[:,1],
                        "label":y})
circles.head()

In [ ]:
plt.scatter(x= circles.X1,y=x[:,1],c=y,cmap=plt.cm.Accent_r)
x.shape, y.shape

In [ ]:
x = torch.from_numpy(x).type(torch.float)
y = torch.from_numpy(y).type(torch.float)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=17)

In [ ]:
import torch
from torch import nn

class CircleBigPro(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 256),
            nn.LayerNorm(256),
            nn.GELU(),
            nn.Dropout(0.2),

            nn.Linear(256, 128),
            nn.LayerNorm(128),
            nn.GELU(),
            nn.Dropout(0.2),

            nn.Linear(128, 64),
            nn.LayerNorm(64),
            nn.GELU(),
            nn.Dropout(0.1),

            nn.Linear(64, 32),
            nn.LayerNorm(32),
            nn.GELU(),
            nn.Dropout(0.1),

            nn.Linear(32, 1)
        )
        
    def forward(self, x):
        return self.model(x)
model_0 = CircleBigPro()

optimizer = torch.optim.AdamW(model_0.parameters(), lr=1e-3, weight_decay=1e-4)

pos_weight = (y_train.shape[0] - y_train.sum()) / y_train.sum()
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

In [ ]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

In [ ]:
model_0.eval()

with torch.inference_mode():
    y_logits = model_0(x_test)[:3000]
y_preds_probs = torch.sigmoid(y_logits)

In [ ]:
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)

best_test_loss = float('inf')
patience = 100 
patience_counter = 0

epochs = 30000
for epoch in range(epochs):
    model_0.train()
    
    y_logits = model_0(x_train).squeeze(dim=-1)
    y_pred = torch.round(torch.sigmoid(y_logits))
    loss = loss_fn(y_logits, y_train)
    
    acc = accuracy_fn(y_true=y_train, y_pred=y_pred)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    scheduler.step(epoch + (epoch % epochs))

    model_0.eval()
    with torch.inference_mode():
        test_logits = model_0(x_test).squeeze(dim=-1)
        test_pred = torch.round(torch.sigmoid(test_logits))
        test_loss = loss_fn(test_logits, y_test)
        test_acc = accuracy_fn(y_true=y_test, y_pred=test_pred)
    
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        patience_counter = 0  
        torch.save(model_0.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1
    
    if patience_counter >= patience:
        print(f"Early stopping at epoch {epoch} due to no improvement.")
        break

    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f} | Acc: {acc:.2f}% | Test Loss: {test_loss:.5f} | Test Acc: {test_acc:.2f}%")


In [ ]:
import requests
from pathlib import Path

if Path("helper_function.py").is_file():
  print("helper_functions.py already exists,skip download")
else:
  print("Downloading helperfunction") 
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open("helper_functions.py","wb") as f:
    f.write(request.content)

In [ ]:
from helper_functions import plot_predictions, plot_decision_boundary

plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.title("Train")
plot_decision_boundary(model_0,x_train,y_train)
plt.subplot(1,2,2)
plt.title("Test")
plot_decision_boundary(model_0,x_test,y_test)